In [391]:
import cPickle
import matplotlib.pyplot as plt
import numpy as np
from progress.bar import Bar
np.random.seed(1234)

In [390]:
with open('/media/Data/flipvanrijn/datasets/coco/context/coco_train_context.pkl', 'r') as f:
    titles = cPickle.load(f)
    descriptions = cPickle.load(f)
    tags = cPickle.load(f)

with open('/media/Data/flipvanrijn/datasets/coco/coco_align.train.pkl', 'rb') as f:
    captions = cPickle.load(f)

with open('/media/Data/flipvanrijn/datasets/coco/images/train2014list.txt', 'r') as f:
    images = f.read().split()

In [3]:
im2idx = {}
for i, data in enumerate(captions):
    cap, im = data
    if im in im2idx:
        im2idx[im].append(i)
    else:
        im2idx[im] = [i]

In [437]:
import gensim
model = gensim.models.Word2Vec.load_word2vec_format('/media/Data/flipvanrijn/models/word2vec/enwiki-latest-pages.bin', binary=True)

In [415]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop = stopwords.words('english')
stemmer = WordNetLemmatizer()

def preprocess(s, model):
    global stop, stemmer
    
    ns = []
    for w in s:
        w = stemmer.lemmatize(w)
        if w in model and w not in stop:
            ns.append(w)
        
    return ns

def sim(s1, s2, model):
    '''
    m = np.zeros((len(s1), len(s2)))
    for ii in xrange(len(s1)):
        for jj in xrange(len(s2)):
            if s1[ii] in normalized_vocab and s2[jj] in normalized_vocab:
                s1i = normalized_vocab[s1[ii]]
                s2j = normalized_vocab[s2[jj]]
                m[ii][jj] = model.similarity(s1i, s2j)
    return (m.sum() / np.count_nonzero(m))
    '''
    s1 = preprocess(s1, model)
    s2 = preprocess(s2, model)
    similarity = model.n_similarity(s1, s2)

    return np.sum(similarity)

def n_relevant(caption, title, description, tags, model, n=10):
    scores = []
    w_caption = caption.split(' ')
    w_title   = title.split(' ')
    if title:
        try:
            scores.append((sim(w_caption, w_title, model), title))
        except:
            print w_title
            raise
    for desc in description.split('.'):
        w_desc = desc.split(' ')
        if desc:
            scores.append((sim(w_caption, w_desc, model), desc))
    if tags:
        scores.append((sim(w_caption, tags, model), ' '.join(tags)))
    
    df = pd.DataFrame(scores, columns=['score', 'sentence'])
    df = df.dropna()
    df = df.sort(['score'], ascending=False)[:n]
    return df['score']

def n_relevant_words(caption, title, description, tags, model, n=100):
    def relevant(s1, s2, model):
        m = np.zeros((len(s1), len(s2)))
        for ii in xrange(len(s1)):
            for jj in xrange(len(s2)):
                if s1[ii] in model and s2[jj] in model:
                    m[ii][jj] = model.similarity(s1[ii], s2[jj])
        scores = []
        for r, c in zip(np.argmax(m, axis=0), range(len(s2))):
            scores.append((m[r][c], s2[c]))
        return scores
    
    cap = preprocess(caption, model)
    title = preprocess(title, model)
    descs = description.split('.')
    ts = preprocess(tags, model)

    scores = []
    scores += relevant(cap, title, model)
    for desc in descs:
        desc = preprocess(desc.split(' '), model)
        scores += relevant(cap, desc, model)
    scores += relevant(cap, ts, model)

    df = pd.DataFrame(scores, columns=['score', 'word'])
    df = df.sort(['score'], ascending=False)
    df = df.dropna()
    df = df.drop_duplicates()
    return df['word'][:n]

In [385]:
np.random.seed(1234)
lengths = []
for i in range(len(titles)):
    caption_idx = np.random.choice(im2idx[i])
    cap = captions[caption_idx][0].split(' ')
    title = titles[i].split(' ')
    desc = descriptions[i]
    ts = tags[i]

    relevant_words = n_relevant_words(cap, title, desc, ts, model)
    lengths.append(relevant_words.shape[0])
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000


In [435]:
from scipy.sparse import csr_matrix, vstack

images_with_context = []
for i, im in enumerate(images):
    caption_idx = np.random.choice(im2idx[i])
    cap = captions[caption_idx][0].split(' ')
    title = titles[i].split(' ')
    desc = descriptions[i]
    ts = tags[i]

    relevant_words = n_relevant_words(cap, title, desc, ts, model)
    
    if len(relevant_words) > 0:
        images_with_context.append(im)
        
        context = np.zeros((100, 300), dtype=np.float32)
        for wi, w in enumerate(relevant_words.values):
            context[wi,:] = model[w]

        if i == 0:
            feat_flatten = csr_matrix(context.flatten())
        else:
            feat_flatten = vstack([feat_flatten, csr_matrix(context.flatten())])
    
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


KeyboardInterrupt: 

(<99x30000 sparse matrix of type '<type 'numpy.float32'>'
 	with 561600 stored elements in Compressed Sparse Row format>,
 ['COCO_train2014_000000000009.jpg',
  'COCO_train2014_000000000025.jpg',
  'COCO_train2014_000000000030.jpg',
  'COCO_train2014_000000000034.jpg',
  'COCO_train2014_000000000036.jpg',
  'COCO_train2014_000000000049.jpg',
  'COCO_train2014_000000000061.jpg',
  'COCO_train2014_000000000064.jpg',
  'COCO_train2014_000000000071.jpg',
  'COCO_train2014_000000000072.jpg',
  'COCO_train2014_000000000077.jpg',
  'COCO_train2014_000000000078.jpg',
  'COCO_train2014_000000000081.jpg',
  'COCO_train2014_000000000086.jpg',
  'COCO_train2014_000000000089.jpg',
  'COCO_train2014_000000000092.jpg',
  'COCO_train2014_000000000094.jpg',
  'COCO_train2014_000000000109.jpg',
  'COCO_train2014_000000000110.jpg',
  'COCO_train2014_000000000113.jpg',
  'COCO_train2014_000000000127.jpg',
  'COCO_train2014_000000000138.jpg',
  'COCO_train2014_000000000142.jpg',
  'COCO_train2014_000000000

In [339]:
scores = []

for i in xrange(len(titles)):
    caption_idx = np.random.randint(len(im2idx[i]))
    caption = captions[caption_idx][0]
    scores.append(n_relevant(caption, titles[i], descriptions[i], tags[i], model))
    if i % 1000 == 0:
        print i


0
1000
2000
3000
4000
5000
6000


KeyboardInterrupt: 

In [77]:
from nltk.stem import WordNetLemmatizer, LancasterStemmer, SnowballStemmer
from nltk.corpus import wordnet

wnl = WordNetLemmatizer()
lst = LancasterStemmer()
sst = SnowballStemmer("english")
print wnl.lemmatize('tables')
print lst.stem('preparing')
model.similarity('kitchen', 'cake')

a = wordnet.synsets('dessert')[0]
b = wordnet.synsets('cakes')[0]
c = wordnet.synset('prepare.v.03')
d = wordnet.synset('frosting.n.01')
print b.path_similarity(a), c.path_similarity(a), d.path_similarity(b)


table
prep
0.0833333333333 0.0909090909091 0.142857142857


In [137]:
summed_scores = []
for s in scores:
    if len(s) > 0:
        summed_scores.append(sum(s) / len(s))

import matplotlib.pyplot as plt
plt.hist(summed_scores)
plt.show()

In [131]:
scores_mismatch = []
for i in xrange(5000):
    choice_idxs = range(len(titles))
    del choice_idxs[i] # remove current
    caption_idx = np.random.randint(len(im2idx[np.random.choice(choice_idxs)]))
    caption = captions[caption_idx][0]
    scores_mismatch.append(n_relevant(caption, titles[i], descriptions[i], tags[i], model))
    if i % 1000 == 0:
        print i

0
1000
2000
3000
4000


In [138]:
summed_scores_mismatch = []
for s in scores_mismatch:
    if len(s) > 0:
        summed_scores_mismatch.append(sum(s) / len(s))

import matplotlib.pyplot as plt
plt.hist(summed_scores_mismatch)
plt.hist(summed_scores)
plt.show()

In [139]:
plt.scatter(range(len(summed_scores)), summed_scores)
plt.scatter(range(len(summed_scores_mismatch)), summed_scores_mismatch)
plt.show()

In [202]:
# context size
sizes = []
for i in xrange(len(titles)):
    count = 0
    if titles[i]:
        count += len(titles[i].split(' '))
    if descriptions[i]:
        for s in descriptions[i].split('.'):
            for w in s.split(" "):
                count += 1
    if tags[i]:
        count += len(tags[i])
    sizes.append(count)

In [204]:
plt.hist(sizes, bins=100, range=(0, 100), cumulative=False)
plt.show()

5


In [217]:
sum(sizes)

2359908